### Quickstart: Compare runs, choose a model, and deploy it to a REST API

In this quickstart, you will:

- Run a hyperparameter sweep on a training script

- Compare the results of the runs in the MLflow UI

- Choose the best run and register it as a model

- Deploy the model to a REST API

- Build a container image suitable for deployment to a cloud platform

In [1]:
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from hyperopt import STATUS_OK, Trials,fmin, hp, tpe

import mlflow
from mlflow.models import infer_signature

In [2]:
data=pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";",
)
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [3]:
data.head(7)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
5,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
6,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,9.6,6


In [ ]:
## Splitting the data inot trainig, Validation and Test sets

## Splitting into training and test sets
train, test= train_test_split(data, test_size=0.2, random_state=42)

train

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
4665,7.3,0.17,0.36,8.20,0.028,44.0,111.0,0.99272,3.14,0.41,12.4,6
1943,6.3,0.25,0.44,11.60,0.041,48.0,195.0,0.99680,3.18,0.52,9.5,5
3399,5.6,0.32,0.33,7.40,0.037,25.0,95.0,0.99268,3.25,0.49,11.1,6
843,6.9,0.19,0.35,1.70,0.036,33.0,101.0,0.99315,3.21,0.54,10.8,7
2580,7.7,0.30,0.26,18.95,0.053,36.0,174.0,0.99976,3.20,0.50,10.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
4426,6.2,0.21,0.52,6.50,0.047,28.0,123.0,0.99418,3.22,0.49,9.9,6
466,7.0,0.14,0.32,9.00,0.039,54.0,141.0,0.99560,3.22,0.43,9.4,6
3092,7.6,0.27,0.52,3.20,0.043,28.0,152.0,0.99129,3.02,0.53,11.4,6
3772,6.3,0.24,0.29,13.70,0.035,53.0,134.0,0.99567,3.17,0.38,10.6,6


In [5]:
train_x = train.drop(columns=['quality'], axis=1).values
train_y = train[['quality']].values.ravel()

## Test dataset
test_x = test.drop(columns=['quality'], axis=1).values
test_y = test[['quality']].values.ravel()

## Splitting the training set into training and validation sets
train_x, valid_x, train_y, valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

signature = infer_signature(train_x, train_y)

In [8]:
np.mean(train_x, axis=0)

array([6.87086790e+00, 2.77890874e-01, 3.33854499e-01, 6.36080089e+00,
       4.55105297e-02, 3.49082642e+01, 1.37556637e+02, 9.94029683e-01,
       3.19041800e+00, 4.89722399e-01, 1.05203340e+01])

In [10]:
import mlflow.tensorflow


def train_model(params, epochs, train_x, train_y, valid_x, valid_y, test_x, test_y):

    # define model architecture
    mean = np.mean(train_x, axis=0) 
    var = np.var(train_x, axis=0)

    model = keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean, variance=var),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )

    ## Compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],
        momentum=params["momentum"],
    ),
    loss = "mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the model with LR and momentum params with mlflow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x, train_y, 
                  validation_data=(valid_x, valid_y), 
                  epochs=epochs,
                  batch_size=params["batch_size"]
                )
        
        ## Evaluate the model 
        eval_res = model.evaluate(valid_x, 
                                  valid_y, 
                                  batch_size=64
                                )
        
        eval_rmse = eval_res[1]

        ## Log the paramters and results to mlflow
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse", eval_rmse)

        ## Log the model with signature and input example
        mlflow.tensorflow.log_model(
            model,"model",
            signature=signature,
        )

        return {
            "loss": eval_rmse,
            "status": STATUS_OK,
            "model": model,
        }

In [11]:
def objective(params):
    ## MLFlow will track the params and results for each run
    result = train_model(
        params, 
        epochs=10,
        train_x= train_x,
        train_y= train_y, 
        valid_x = valid_x, 
        valid_y = valid_y, 
        test_x = test_x, 
        test_y = test_y
    )

    return result

In [15]:
space = {
    'lr': hp.loguniform('lr', np.log(1e-5), np.log(1e-1)),
    'momentum' : hp.uniform('momntum', 0.0, 1.0),
    'batch_size': hp.choice('batch_size', [32, 64, 128])
}

In [16]:
mlflow.set_experiment("/wine quality")
with mlflow.start_run():
    ## COnduct a hyper parameter search with hyperopt
    trials = Trials()
    best = fmin(
        fn = objective,
        space = space,
        algo =  tpe.suggest,
        max_evals = 4,
        trials = trials,
    )

    ## Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse", best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"], "model", signature=signature)

    # Print out the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}") 


Epoch 1/10                                           

 1/49 ━━━━━━━━━━━━━━━━━━━━ 28s 592ms/step - loss: 36.0781 - root_mean_squared_error: 6.0065
26/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 33.8845 - root_mean_squared_error: 5.8206   
49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 32.9561 - root_mean_squared_error: 5.7398 - val_loss: 28.5451 - val_root_mean_squared_error: 5.3428

Epoch 2/10                                           

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 27.0900 - root_mean_squared_error: 5.2048
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 26.6014 - root_mean_squared_error: 5.1572 
49/49 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 26.5762 - root_mean_squared_error: 5.1548 - val_loss: 23.1211 - val_root_mean_squared_error: 4.8084

Epoch 3/10                                           

 1/49 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 21.7282 - root_mean_squared_error: 4.6613
40/49 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 21.5451 - root_mean_squared_error: 4.6

In [ ]:
# ## Inference with the best model

# import mlflow

# model_uri = 'runs:/d442b2d7b6a44ecea998b970a3332f2b/model'

# # Replace INPUT_EXAMPLE with your own input example to the model
# # A valid input example is a data instance suitable for pyfunc prediction
# input_data = test_x

# # Verify the model with the provided input data using the logged dependencies.
# # For more details, refer to:
# # https://mlflow.org/docs/latest/models.html#validate-models-before-deployment
# mlflow.models.predict(
#     model_uri=model_uri,
#     input_data=input_data,
#     env_manager="uv",
# )